# BIP's error function

To the error function does expected to receive as input the following pandas.DataFrame struncture:

| StoreID  | Month | Region | NumberOfSales | _NumberOfSales |
| ---------|----------------|---------------| ---------------|
| 1000	   |3      | 4      | 16            | 16             |
| 1000	   |4      | 4      | 30            | 23             |
| 1001	   |3      | 6      | 410           | 411            |
| 1001	   |4      | 27     | 3130          | 3120           |
| 1002	   |3      | 58     | 10            | 8              |

Where:
 
 - *NumberOfSales* are the **test actual values**
 - *_NumberOfSales* are **the predicted values**
 
 

Start from test set to simulate a predicted dataset

In [1]:
from import_man import *

df = pd.read_csv('./dataset/preprocessed_train.csv')

print("Shape before: " + str(df.shape))

# Let's work on a reduced instance of the test set
df = df.sample(n=5000)

print("Shape after: " + str(df.shape))

Shape before: (523021, 51)
Shape after: (5000, 51)


In [2]:
error_evaluation_columns = ['StoreID', 'D_Month', 'Region', 'NumberOfSales', '_NumberOfSales']

# Create fake predicted sales 
df['_NumberOfSales'] = df.NumberOfSales.apply(lambda x: df['NumberOfSales'].sample().values[0])


# Save a copy of the dataframe to be passed to the function in order to evaluate it
df_BIP_err_fun_eval = df


# Remove useless columns
df =  df[error_evaluation_columns]

df.head(20)

,StoreID,D_Month,Region,NumberOfSales,_NumberOfSales
210860,1301,11,2,5104.811659,2959.000000
227093,1323,12,10,9602.000000,3421.000000
376935,1539,9,5,2329.669983,4513.000000
463407,1664,1,0,5007.000000,4332.998322
205369,1293,10,7,4277.000000,2966.000000
470317,1674,7,0,5739.000000,4912.837209
184238,1263,4,3,5637.000000,3525.000000
362601,1519,11,9,2085.000000,6723.000000
470865,1675,1,2,3454.000000,3182.291874
463235,1664,8,0,2993.000000,6201.000000


In [3]:
# let's keep only march and april
# df = df.loc[df['D_Month'].isin([3, 4])] No. let's just evaluate months provided

# compute the difference between actual and predicted NumberOfSales and do the abs
df['abs_diff'] = df.apply(lambda x: abs(x['NumberOfSales'] - x['_NumberOfSales']), axis=1)

df.head(20)

,StoreID,D_Month,Region,NumberOfSales,_NumberOfSales,abs_diff
210860,1301,11,2,5104.811659,2959.000000,2145.811659
227093,1323,12,10,9602.000000,3421.000000,6181.000000
376935,1539,9,5,2329.669983,4513.000000,2183.330017
463407,1664,1,0,5007.000000,4332.998322,674.001678
205369,1293,10,7,4277.000000,2966.000000,1311.000000
470317,1674,7,0,5739.000000,4912.837209,826.162791
184238,1263,4,3,5637.000000,3525.000000,2112.000000
362601,1519,11,9,2085.000000,6723.000000,4638.000000
470865,1675,1,2,3454.000000,3182.291874,271.708126
463235,1664,8,0,2993.000000,6201.000000,3208.000000


In [4]:
# Let's sum over the region
df_sums_by_region = df.groupby(['Region']).sum()
df_sums_by_region.head(20)

,StoreID,D_Month,NumberOfSales,_NumberOfSales,abs_diff
Region,,,,,
0,519576,2457,1.742487e+06,1.748741e+06,8.335454e+05
1,215313,1111,7.584487e+05,8.134647e+05,3.619951e+05
2,904499,3740,3.074894e+06,3.167847e+06,1.285880e+06
3,893224,4331,3.770968e+06,3.103745e+06,1.670652e+06
4,175467,865,7.631371e+05,6.205706e+05,3.905720e+05
5,412490,2120,1.448195e+06,1.542033e+06,7.218206e+05
6,313347,1535,9.715543e+05,1.161763e+06,4.647186e+05
7,606479,2865,2.092793e+06,2.081793e+06,9.582101e+05
8,254404,1190,7.794823e+05,9.975703e+05,3.952301e+05


In [5]:
# Divide the difference between actual and predicted NumberOfSales by the sum of actual
df_sums_by_region['E_r'] = df_sums_by_region['abs_diff'] / df_sums_by_region['NumberOfSales']

df_sums_by_region.head(20)

,StoreID,D_Month,NumberOfSales,_NumberOfSales,abs_diff,E_r
Region,,,,,,
0,519576,2457,1.742487e+06,1.748741e+06,8.335454e+05,0.478365
1,215313,1111,7.584487e+05,8.134647e+05,3.619951e+05,0.477284
2,904499,3740,3.074894e+06,3.167847e+06,1.285880e+06,0.418187
3,893224,4331,3.770968e+06,3.103745e+06,1.670652e+06,0.443030
4,175467,865,7.631371e+05,6.205706e+05,3.905720e+05,0.511798
5,412490,2120,1.448195e+06,1.542033e+06,7.218206e+05,0.498428
6,313347,1535,9.715543e+05,1.161763e+06,4.647186e+05,0.478325
7,606479,2865,2.092793e+06,2.081793e+06,9.582101e+05,0.457862
8,254404,1190,7.794823e+05,9.975703e+05,3.952301e+05,0.507042


In [6]:
# Get the number of regions
N_regions = len(df.Region.unique())

print("Number of regions: {}".format(N_regions))

Number of regions: 11


In [7]:
Error = df_sums_by_region['E_r'].sum() / N_regions

print("BIP error: {}".format(Error))

BIP error: 0.4688234560618476


### Test of the BIP error function implemented

Test that the get_BIP_error function works as expected

In [8]:
from BIP_error import get_BIP_error

df_BIP_err_fun_eval.head()

,StoreID,Date,IsHoliday,IsOpen,HasPromotions,StoreType,AssortmentType,NearestCompetitor,Region,NumberOfCustomers,...,StoreType_ShoppingCenter,AssortmentType_General,AssortmentType_WithNFDept,AssortmentType_WithFishDept,Events_Fog,Events_Hail,Events_Rain,Events_Snow,Events_Thunderstorm,_NumberOfSales
210860,1301,20/11/2016,0,0,0,Standard Market,With Non-Food Department,8389,2,226.706278,...,0,0,1,0,0,0,0,0,0,2959.000000
227093,1323,04/12/2017,0,1,1,Hyper Market,With Non-Food Department,4154,10,640.000000,...,0,0,1,0,0,0,0,0,0,3421.000000
376935,1539,17/09/2017,0,0,0,Super Market,With Non-Food Department,19821,5,125.595357,...,0,0,1,0,0,0,1,0,0,4513.000000
463407,1664,23/01/2017,0,1,1,Hyper Market,General,1057,0,307.000000,...,0,1,0,0,1,0,1,0,0,4332.998322
205369,1293,27/10/2017,0,1,1,Standard Market,With Non-Food Department,6330,7,220.000000,...,0,0,1,0,1,0,0,0,0,2966.000000


In [9]:
# use the function
error = get_BIP_error(df_BIP_err_fun_eval)

Number of regions identified: 11
BIP total error: 0.4688234560618476
